# Forecasting Net Prophet

You’re a growth analyst at [MercadoLibre](http://investor.mercadolibre.com/investor-relations). With over 200 million users, MercadoLibre is the most popular e-commerce site in Latin America. You've been tasked with analyzing the company's financial and user data in clever ways to make the company grow. So, you want to find out if the ability to predict search traffic can translate into the ability to successfully trade the stock.

Instructions

This section divides the instructions for this Challenge into four steps and an optional fifth step, as follows:

* Step 1: Find unusual patterns in hourly Google search traffic

* Step 2: Mine the search traffic data for seasonality

* Step 3: Relate the search traffic to stock price patterns

* Step 4: Create a time series model with Prophet

* Step 5 (optional): Forecast revenue by using time series models

The following subsections detail these steps.

## Step 1: Find Unusual Patterns in Hourly Google Search Traffic

The data science manager asks if the Google search traffic for the company links to any financial events at the company. Or, does the search traffic data just present random noise? To answer this question, pick out any unusual patterns in the Google search data for the company, and connect them to the corporate financial events.

To do so, complete the following steps:

1. Read the search data into a DataFrame, and then slice the data to just the month of May 2020. (During this month, MercadoLibre released its quarterly financial results.) Use hvPlot to visualize the results. Do any unusual patterns exist?

2. Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?

## Step 2: Mine the Search Traffic Data for Seasonality

Marketing realizes that they can use the hourly search data, too. If they can track and predict interest in the company and its platform for any time of day, they can focus their marketing efforts around the times that have the most traffic. This will get a greater return on investment (ROI) from their marketing budget.

To that end, you want to mine the search traffic data for predictable seasonal patterns of interest in the company. To do so, complete the following steps:

1. Group the hourly search data to plot the average traffic by the day of the week (for example, Monday vs. Friday).

2. Using hvPlot, visualize this traffic as a heatmap, referencing the `index.hour` as the x-axis and the `index.dayofweek` as the y-axis. Does any day-of-week effect that you observe concentrate in just a few hours of that day?

3. Group the search data by the week of the year. Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?

## Step 3: Relate the Search Traffic to Stock Price Patterns

You mention your work on the search traffic data during a meeting with people in the finance group at the company. They want to know if any relationship between the search data and the company stock price exists, and they ask if you can investigate.

To do so, complete the following steps:

1. Read in and plot the stock price data. Concatenate the stock price data to the search data in a single DataFrame.

2. Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Slice the data to just the first half of 2020 (`2020-01` to `2020-06` in the DataFrame), and then use hvPlot to plot the data. Do both time series indicate a common trend that’s consistent with this narrative?

3. Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

    * “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

    * “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis

4. Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

## Step 4: Create a Time Series Model with Prophet

Now, you need to produce a time series model that analyzes and forecasts patterns in the hourly search data. To do so, complete the following steps:

1. Set up the Google search data for a Prophet forecasting model.

2. After estimating the model, plot the forecast. How's the near-term forecast for the popularity of MercadoLibre?

3. Plot the individual time series components of the model to answer the following questions:

    * What time of day exhibits the greatest popularity?

    * Which day of the week gets the most search traffic?

    * What's the lowest point for search traffic in the calendar year?

## Step 5 (Optional): Forecast Revenue by Using Time Series Models

A few weeks after your initial analysis, the finance group follows up to find out if you can help them solve a different problem. Your fame as a growth analyst in the company continues to grow!

Specifically, the finance group wants a forecast of the total sales for the next quarter. This will dramatically increase their ability to plan budgets and to help guide expectations for the company investors.

To do so, complete the following steps:

1. Read in the daily historical sales (that is, revenue) figures, and then apply a Prophet model to the data.

2. Interpret the model output to identify any seasonal patterns in the company's revenue. For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

3. Produce a sales forecast for the finance group. Give them a number for the expected total sales in the next quarter. Include the best- and worst-case scenarios to help them make better plans.


## Install and import the required libraries and dependencies

<a href="https://github.com/cannabbeers/11_Google_search_traffic_analysis_for_MercadoLibre/blob/main/forecasting_net_prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# # Our instructor said we can skip the google colab piece if on windows
# # however, before he told us it was ok to skip, I found some stuff like the 'open in colab' button while trying to figure out colab
# # see code reference in readme for the fancy colab button and this may even work in colab if you unhash all colab dependencies
# # have not tested and disabled since causing problems with my matplolib inline, (i think) 

# # Install the required libraries
# !pip install pystan
# !pip install prophet
# !pip install hvplot
# !pip install holoviews

In [ ]:
# # Import the required libraries and dependencies

import numpy as np
import pandas as pd
import holoviews as hv
import datetime as dt
import hvplot.pandas

from prophet import Prophet

%matplotlib inline

In [ ]:
# Imports for every notebook

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
from watermark import watermark
print(watermark())
print(watermark(iversions=True, globals_=globals(), packages=""))

## Step 1: Find Unusual Patterns in Hourly Google Search Traffic

The data science manager asks if the Google search traffic for the company links to any financial events at the company. Or, does the search traffic data just present random noise? To answer this question, pick out any unusual patterns in the Google search data for the company, and connect them to the corporate financial events.

To do so, complete the following steps:

1. Read the search data into a DataFrame, and then slice the data to just the month of May 2020. (During this month, MercadoLibre released its quarterly financial results.) Use hvPlot to visualize the results. Do any unusual patterns exist?

2. Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?


#### Step 1: Read the search data into a DataFrame, and then slice the data to just the month of May 2020. (During this month, MercadoLibre released its quarterly financial results.) Use hvPlot to visualize the results. Do any unusual patterns exist?

In [ ]:
# Upload the "google_hourly_search_trends.csv" file into Colab, then store in a Pandas DataFrame
# Set the "Date" column as the Datetime Index.

# from google.colab import files
# uploaded = files.upload()

In [ ]:
df_mercado_trends = pd.read_csv(
    "Resources/google_hourly_search_trends.csv", 
    index_col='Date',     
    parse_dates=True, 
    infer_datetime_format=True
)

# Review the first and last five rows of the DataFrame
df_mercado_trends

In [ ]:
# Review the data types of the DataFrame using the info function

df_mercado_trends.info()

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Slice the DataFrame to just the month of May 2020
df_may_2020 = df_mercado_trends.loc['2020-05']

# Use hvPlot to visualize the data for May 2020
MAY_line_plot = df_may_2020.hvplot(
    title='Hourly Google Search Traffic Data: May 2020', 
    xlabel='Date', 
    ylabel='Hourly Searches', 
    width=500,
    height=250
)

# visuals will display later in-line with answers to challenge questions

In [ ]:
# created box-plot as additional option to visualize range and outliers
# visuals will display later in-line with answers to challenge questions

MAY_box_plot = df_may_2020.hvplot.box(
    title='Hourly Google Search Traffic Data Box Plot: May 2020',
    xlabel='Full Month of May 2020',
    ylabel='Search Trends',
    width=500,
    height=250
)

In [ ]:
# created scatter-plot as additional option to visualize range and outliers
# visuals will display later in-line with answers to challenge questions

MAY_scatter_plot = df_may_2020.hvplot.scatter(
    x='Date',
    y='Search Trends',
    title='Scatter Plot of Google Search Trends',
    xlabel='Date',
    ylabel='Hourly Search Plots',
    width=500,
    height=500, 
    yticks=24,
    invert=True, 
    grid=True # added gridlines to make visual more useful, easier to read
    
)

**Question:** Do any unusual patterns exist?

**Answer:** Yes, we see around May 5, there is a spike in traffic from the most basic `hvplot`.  

Then we look at a box plot to see how far away the peaks look compared to rest of the data.

In [ ]:
MAY_line_plot + MAY_box_plot

### The `box_plot` looks like this is worth exploring further, so now zooming in to an hourly traffic scatter_plot
Notice that in the `scatter_plot`, each dot represents one hour and you can check values easily when hover over point.

In [ ]:
MAY_scatter_plot

This reveals there were only 6 hours in May where the search count was above 100, all on Cinco de Mayo.

## Step 2: Calculate the total search traffic for the month, and then compare the value to the monthly median across all months. Did the Google search traffic increase during the month that MercadoLibre released its financial results?

In [ ]:
df_may_2020.info()

In [ ]:
# Calculate the sum of the total search traffic for May 2020

# since the df has only one column plus the index, we can simply call the sum of df_may_2020
# `traffic_may_2020 = df_may_2020.sum()`
# this is how we would code to call the sum of items in "Search Trends" column 

traffic_may_2020 = df_may_2020['Search Trends'].sum()

# View the traffic_may_2020 value

print(f'{traffic_may_2020} - total search count for May 2020')

In [ ]:
# Calcluate the monthly median search traffic across all months 
# Group the DataFrame by index year and then index month, chain the sum and then the median functions

median_monthly_traffic = df_mercado_trends['Search Trends'].groupby(
    [df_mercado_trends.index.year, df_mercado_trends.index.month]).sum().median()

print(f'{median_monthly_traffic} - median monthly total search traffic')

### Found alternative approach to get same output of 35172.51 for the monthly mean using `.resample` 
this method also returns the data type with the value
+ [pandas.DataFrame.resample](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html)
+ [Time Series DataFrames in Pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/09_timeseries.html)

In [ ]:
# pandas.DataFrame.resample

median_monthly_sums_df = df_mercado_trends.resample('M').sum().median()

# View the median_monthly_traffic value
median_monthly_sums_df

In [ ]:
# Compare the search traffic for May 2020 to the overall monthly median value
May_2020_vs_monthly_median_plot = pd.DataFrame({
    'Month': ['May 2020', 'Overall Monthly Median'],
    'Search Traffic': [traffic_may_2020, median_monthly_sums_df.values]
}).hvplot.bar(
    x='Month',
    y='Search Traffic',
    title='ML Search Traffic: Overall Monthly Median vs. May 2020',
    xlabel='',
    ylabel='Google Search Traffic',
    rot=45,
    ylim=(33500,38500),
    width=500,          
    height=250
)
May_2020_vs_monthly_median_plot



##### Answer the following question: 

**Question:** Did the Google search traffic increase during the month that MercadoLibre released its financial results?

**Answer:** Yes, if they released their financials in May 2020...

In [ ]:
# Calculate the percentage difference between May 2020 traffic and overall monthly median traffic
delta = ((traffic_may_2020 - median_monthly_traffic) / median_monthly_traffic) * 100

delta

In [ ]:
print(f'{(delta):.2f}% increase in search traffic in May, based on our earlier `hvplot` visuals, May 5th "spike" generated the anomoly')

## Step 2: Mine the Search Traffic Data for Seasonality

Marketing realizes that they can use the hourly search data, too. If they can track and predict interest in the company and its platform for any time of day, they can focus their marketing efforts around the times that have the most traffic. This will get a greater return on investment (ROI) from their marketing budget.

To that end, you want to mine the search traffic data for predictable seasonal patterns of interest in the company. To do so, complete the following steps:

1. Group the hourly search data to plot the average traffic by the day of the week (for example, Monday vs. Friday).

2. Using hvPlot, visualize this traffic as a heatmap, referencing the `index.hour` as the x-axis and the `index.dayofweek` as the y-axis. Does any day-of-week effect that you observe concentrate in just a few hours of that day?

3. Group the search data by the week of the year. Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?


#### Step 1: Group the hourly search data to plot the average traffic by the day of the week (for example, Monday vs. Friday).

In [ ]:
# define the list 'day_of_week' to enhance visuals with and substitute "day_of_week" for their indexed number position
# tuples allow us to modify day of the week axis tickers to actual days instead of numbered positions
# once created, we can re-use anywhere after this cell

day_of_week = [(0, 'Monday'), (1, 'Tuesday'), (2, 'Wednesday'), (3, 'Thursday'), (4, 'Friday'), (5, 'Saturday'), (6, 'Sunday')]


In [ ]:
# Calcluate the monthly median search traffic across all months 
# Group the DataFrame by index year and then index month, chain the median function

hv.extension('bokeh')

# Group the hourly search data to plot (use hvPlot) the average traffic by the day of week 
# using 
df_mercado_trends.groupby(by=[df_mercado_trends.index.day_of_week]).mean().hvplot(
    title='"Average Google Search Traffic by "Day of the Week"',
    xlabel='Day of the Week',
    xticks=day_of_week,
    frame_width=500,
    frame_height=250,
    rot=45
)

#### Step 2: Using hvPlot, visualize this traffic as a heatmap, referencing the `index.hour` as the x-axis and the `index.dayofweek` as the y-axis. Does any day-of-week effect that you observe concentrate in just a few hours of that day?

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the hour of the day and day of week search traffic as a heatmap.
plasma_heat_map_plot = df_mercado_trends.hvplot.heatmap(
    x='index.hour',
    y='index.dayofweek',
    xlabel='24 Hour Clock',
    xticks=list(range(0, 24, 2)),
    ylabel='Day of the Week',
    yticks=day_of_week,
    C='Search Trends',
    clabel='Search Count',
    cmap='plasma',
    frame_width=500,
    frame_height=250
).aggregate(function=np.mean)

plasma_heat_map_plot

# Answer the following question:

**Question:** Does any day-of-week effect that you observe concentrate in just a few hours of that day?

**Answer:** it appears each day experiences it's own peak search traffic from about 22:00 to 01:00 hours.  It also appears you can see Monday peaks clearly on hour 23:00, Tuesday through Friday have a wider time frame peaking from 23:00 up untim 02:00.

It also appears the brightest plots on the heatmap indicate highest search volume at 23:00 on Monday then peaking from 0:00 to 01:00 on Tuesday which is consistent with previous line plot for "Days of the Week Search Trends".

#### Step 3: Group the search data by the week of the year. Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Group the hourly search data to plot (use hvPlot) the average traffic by the week of the year
df_mercado_trends.groupby(
    by=[df_mercado_trends.index.weekofyear]).mean().hvplot(
    title='ML Average Weekly Google Search Traffic (52 week calendar)',
    xlabel='Week of the Year',
    frame_width=500,
    frame_height=250
)

##### Answer the following question:

**Question:** Does the search traffic tend to increase during the winter holiday period (weeks 40 through 52)?

**Answer:** # Yes, and since this shows the increase is bouncing up from annual lows and mostly plateaus until a steady decline at the beginning of springtime, this looks to support the case that seasonality impacts search count.

## Step 3: Relate the Search Traffic to Stock Price Patterns

You mention your work on the search traffic data during a meeting with people in the finance group at the company. They want to know if any relationship between the search data and the company stock price exists, and they ask if you can investigate.

To do so, complete the following steps:

1. Read in and plot the stock price data. Concatenate the stock price data to the search data in a single DataFrame.

2. Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Slice the data to just the first half of 2020 (`2020-01` to `2020-06` in the DataFrame), and then use hvPlot to plot the data. Do both time series indicate a common trend that’s consistent with this narrative?

3. Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

    * “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

    * “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis

4. Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?


#### Step 1: Read in and plot the stock price data. Concatenate the stock price data to the search data in a single DataFrame.

In [ ]:
# Upload the "mercado_stock_price.csv" file into Colab, then store in a Pandas DataFrame
# Set the "date" column as the Datetime Index.
# from google.colab import files
# uploaded = files.upload()

df_mercado_stock = pd.read_csv(
    "Resources/mercado_stock_price.csv", 
    index_col='date',
    infer_datetime_format=True, 
    parse_dates=True
)

# View the first and last five rows of the DataFrame
df_mercado_stock

In [ ]:
df_mercado_stock.info()

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the closing price of the df_mercado_stock DataFrame

df_mercado_stock.hvplot(
    title='ML Close Price: 2015-2020', 
    xlabel='Date', 
    ylabel='Closing Price',
    frame_width=500,
    frame_height=250
)


In [ ]:
# Concatenate the df_mercado_stock DataFrame with the df_mercado_trends DataFrame
# Concatenate the DataFrame by columns (axis=1), and drop and rows with only one column of data

mercado_stock_trends_df = pd.concat([df_mercado_stock, df_mercado_trends], axis=1).dropna()

# View the first and last five rows of the DataFrame

mercado_stock_trends_df

#### Step 2: Market events emerged during the year of 2020 that many companies found difficult. But, after the initial shock to global financial markets, new customers and revenue increased for e-commerce platforms. Slice the data to just the first half of 2020 (`2020-01` to `2020-06` in the DataFrame), and then use hvPlot to plot the data. Do both time series indicate a common trend that’s consistent with this narrative?

In [ ]:
# For the combined dataframe, slice to just the first half of 2020 (2020-01 through 2020-06) 

first_half_2020 = mercado_stock_trends_df.loc['2020-01':'2020-06']

# View the first and last five rows of first_half_2020 DataFrame

first_half_2020

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the close and Search Trends data
# Plot each column on a separate axes using the following syntax
# `hvplot(shared_axes=False, subplots=True).cols(1)`

first_half_2020.hvplot(shared_axes=False, subplots=True, rot=45, frame_width=500, frame_height=250).cols(1)



##### Answer the following question:

**Question:** Do both time series indicate a common trend that’s consistent with this narrative?

**Answer:** Yes, it appears that the rise in ML's stock 'close price' in early May corresponds to the exact time when 'Google Search Traffic' spiked on May 5th, likely when earnings or important news was released.  Time series shows us the increasing close price running all the way to the end of the dataframe and then even possibly another smaller spike in search traffic in late June 2020.  

It would be interesting to also compare the RSI of the stock around this time as the curve of the stock price increase also begins to flatten in late June.  If RSI was approaching or past 80, it could have been a technical indicator showing signs the stock was about to be 'overbought'.  From that point it's ready to be singled out as "over-valued" unless ML maintains firm guidance on positive momentum and continued revenue expansion.

#### Step 3: Create a new column in the DataFrame named “Lagged Search Trends” that offsets, or shifts, the search traffic by one hour. Create two additional columns:

* “Stock Volatility”, which holds an exponentially weighted four-hour rolling average of the company’s stock volatility

* “Hourly Stock Return”, which holds the percent change of the company's stock price on an hourly basis


In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Lagged Search Trends
# This column should shift the Search Trends information by one hour

# `.shift(1)` moves the values of the "Search Trends" column one step backward in the DataFrame. 
# thus creating a time lag of one hour for the values held in the new column "Lagged Search Trends"

mercado_stock_trends_df['Lagged Search Trends'] = mercado_stock_trends_df['Search Trends'].shift(1)

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Stock Volatility
# This column should calculate the standard deviation of the closing stock price return data over a 4 period rolling window

# `.pct_change()` calculates the change in stock price from day before
# `.rolling(window=4)` creates a rolling 4-day window
# `.std()` calculates the SD of the % changes within that window as it all moves forward in time

mercado_stock_trends_df['Stock Volatility'] = mercado_stock_trends_df['close'].pct_change().rolling(window=4).std()

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# Use hvPlot to visualize the stock volatility
mercado_stock_trends_df['Stock Volatility'].hvplot(
    title='Volatility in MercadoLibre (MELI) on NASDAQ', 
    frame_width=500, 
    frame_height=250
)

**Solution Note:** Note how volatility spiked, and tended to stay high, during the first half of 2020. This is a common characteristic of volatility in stock returns worldwide: high volatility days tend to be followed by yet more high volatility days. When it rains, it pours.

In [ ]:
# Create a new column in the mercado_stock_trends_df DataFrame called Hourly Stock Return
# This column should calculate hourly return percentage of the closing price

mercado_stock_trends_df['Hourly Stock Return'] = mercado_stock_trends_df['close'].pct_change()

In [ ]:
# View the first and last five rows of the mercado_stock_trends_df DataFrame

mercado_stock_trends_df

#### Step 4: Review the time series correlation, and then answer the following question: Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

In [ ]:
# Construct correlation table of Stock Volatility, Lagged Search Trends, and Hourly Stock Return
mercado_stock_trends_df[['Stock Volatility', 'Lagged Search Trends', 'Hourly Stock Return']].corr()

##### Answer the following question:


**Question:** Does a predictable relationship exist between the lagged search traffic and the stock volatility or between the lagged search traffic and the stock price returns?

**Answer:** based on our data, "Lagged Search Trends" has a positive correlation with "Hourly Stock Return" and a negative correlation with "Stock Volatility", however, both are too weak to call anything predictable given the data reviewed.  

## some more info about the `.corr()` method in pandas...
[pandas.DataFrame.corr](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html#pandas.DataFrame.corr)
[Compute pairwise correlation of columns, excluding NA/null values.](https://github.com/pandas-dev/pandas/blob/v2.0.3/pandas/core/frame.py#L9980-L10092)
+ if no method parameter is chosen, when using `.corr()` then it defaults to [*Pearson's correlation*](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient)
+ we can also apply the *Kendall* or *Spearman* correlations to see if we find any other clue
+ calculating with other methods below shows potentially different correations may exist, however, also too small to become predictable with our current data

In [ ]:
# Pearson Method: default measure of linear correlation (ratio between the covariance of two variables and the product of their standard deviations) 

pearson_matrix = mercado_stock_trends_df[['Stock Volatility', 'Lagged Search Trends', 'Hourly Stock Return']].corr(method='pearson') 


In [ ]:
# measure of the ordinal association between two measured quantities
# "Kendall correlation between two variables will be high when observations have a similar (or identical for a correlation of 1) rank
# (i.e. relative position label of the observations within the variable: 1st, 2nd, 3rd, etc.) between the two variables
# low when observations have a dissimilar (or fully different for a correlation of −1) rank between the two variables"
# https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient

kendall_matrix = mercado_stock_trends_df[['Stock Volatility', 'Lagged Search Trends', 'Hourly Stock Return']].corr(method='kendall')


In [ ]:
# Spearman's coefficient is appropriate for both continuous and discrete ordinal variables
# may find non-linear correlations missed by Pearson correlation
# "Spearman correlation coefficient is defined as the Pearson correlation coefficient between the rank variables" - https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient

spearman_matrix = mercado_stock_trends_df[['Stock Volatility', 'Lagged Search Trends', 'Hourly Stock Return']].corr(method='spearman')

In [ ]:
print(pearson_matrix)
print(kendall_matrix)
print(spearman_matrix) 

## Step 4: Create a Time Series Model with Prophet

Now, you need to produce a time series model that analyzes and forecasts patterns in the hourly search data. To do so, complete the following steps:

1. Set up the Google search data for a Prophet forecasting model.

2. After estimating the model, plot the forecast. How's the near-term forecast for the popularity of MercadoLibre?

3. Plot the individual time series components of the model to answer the following questions:

    * What time of day exhibits the greatest popularity?

    * Which day of the week gets the most search traffic?

    * What's the lowest point for search traffic in the calendar year?


#### Step 1: Set up the Google search data for a Prophet forecasting model.

In [ ]:
# Using the df_mercado_trends DataFrame, reset the index so the date information is no longer the index
mercado_prophet_df = df_mercado_trends.reset_index()

# Label the columns ds and y so that the syntax is recognized by Prophet
mercado_prophet_df.columns = ['ds', 'y']

# Drop an NaN values from the prophet_df DataFrame
mercado_prophet_df = mercado_prophet_df.dropna()

# View the first and last five rows of the mercado_prophet_df DataFrame
mercado_prophet_df

In [ ]:
# Call the Prophet function, store as an object
model_mercado_trends = Prophet()

In [ ]:
# Fit the time-series model.
model_mercado_trends.fit(mercado_prophet_df)

In [ ]:
# Create a future dataframe to hold predictions
# Make the prediction go out as far as 2000 hours (approx 80 days)
future_mercado_trends = model_mercado_trends.make_future_dataframe(periods=2000, freq='H')

# View the last five rows of the future_mercado_trends DataFrame
future_mercado_trends.tail()

In [ ]:
# Make the predictions for the trend data using the future_mercado_trends DataFrame
forecast_mercado_trends = model_mercado_trends.predict(future_mercado_trends)

# Display the first five rows of the forecast_mercado_trends DataFrame
forecast_mercado_trends.head()

#### Step 2: After estimating the model, plot the forecast. How's the near-term forecast for the popularity of MercadoLibre?

In [ ]:
# Plot the Prophet predictions for the Mercado trends data
model_mercado_trends.plot(forecast_mercado_trends)

##### Answer the following question:

**Question:**  How's the near-term forecast for the popularity of MercadoLibre?

**Answer:** Near-term forecast shows some decline in popularity of ML.  If we determined the drop in popularity is temporary or cyclical in nature and a temporary, we could  start to look for chances to accumulate ML if we see a lagging deterioration in stock close price and set price points we are buying based partially on other indicators such as moving averages, bolinger bands, RSI, etc.

#### Step 3: Plot the individual time series components of the model to answer the following questions:

* What time of day exhibits the greatest popularity?

* Which day of the week gets the most search traffic?

* What's the lowest point for search traffic in the calendar year?


In [ ]:
# Set the index in the forecast_mercado_trends DataFrame to the ds datetime column
forecast_mercado_trends = forecast_mercado_trends.set_index('ds')

# View the only the yhat,yhat_lower and yhat_upper columns from the DataFrame
forecast_mercado_trends[['yhat', 'yhat_lower', 'yhat_upper']].head()

Solutions Note: `yhat` represents the most likely (average) forecast, whereas `yhat_lower` and `yhat_upper` represents the worst and best case prediction (based on what are known as 95% confidence intervals).

In [ ]:
# Holoviews extension to render hvPlots in Colab
hv.extension('bokeh')

# From the forecast_mercado_trends DataFrame, use hvPlot to visualize
#  the yhat, yhat_lower, and yhat_upper columns over the last 2000 hours 
forecast_mercado_trends[['yhat', 'yhat_lower', 'yhat_upper']].iloc[-2000:,:].hvplot(
    title='80-Day Forecast of Mercado Libre Google Search Traffic', 
    xlabel='Date',
    ylabel='Search Trends',
    frame_width=5000,
    frame_height=500
)

In [ ]:
# Reset the index in the forecast_mercado_trends DataFrame
forecast_mercado_trends = forecast_mercado_trends.reset_index()

# Use the plot_components function to visualize the forecast results 
# for the forecast_canada DataFrame 
figures_mercado_trends = model_mercado_trends.plot_components(forecast_mercado_trends)

##### Answer the following questions:

**Question:** What time of day exhibits the greatest popularity?

**Answer:** 23:00 to 01:00

**Question:** Which day of week gets the most search traffic? 
   
**Answer:** Tuesday

**Question:** What's the lowest point for search traffic in the calendar year?

**Answer:** mid-October

## Step 5 (Optional): Forecast Revenue by Using Time Series Models

A few weeks after your initial analysis, the finance group follows up to find out if you can help them solve a different problem. Your fame as a growth analyst in the company continues to grow!

Specifically, the finance group wants a forecast of the total sales for the next quarter. This will dramatically increase their ability to plan budgets and to help guide expectations for the company investors.

To do so, complete the following steps:

1. Read in the daily historical sales (that is, revenue) figures, and then apply a Prophet model to the data. The daily sales figures are quoted in millions of USD dollars.

2. Interpret the model output to identify any seasonal patterns in the company's revenue. For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

3. Produce a sales forecast for the finance group. Give them a number for the expected total sales in the next quarter. Include the best- and worst-case scenarios to help them make better plans.




#### Step 1: Read in the daily historical sales (that is, revenue) figures, and then apply a Prophet model to the data.

In [ ]:
# # Upload the "mercado_daily_revenue.csv" file into Colab, then store in a Pandas DataFrame
# # Set the "date" column as the DatetimeIndex
# # Sales are quoted in millions of US dollars
# from google.colab import files
# uploaded = files.upload()

# df_mercado_sales = # YOUR CODE HERE

# # Review the DataFrame
# # YOUR CODE HERE


In [ ]:
# # Holoviews extension to render hvPlots in Colab
# hv.extension('bokeh')

# # Use hvPlot to visualize the daily sales figures 
# # YOUR CODE HERE


In [ ]:
# # Apply a Facebook Prophet model to the data.

# # Set up the dataframe in the neccessary format:
# # Reset the index so that date becomes a column in the DataFrame
# mercado_sales_prophet_df = # YOUR CODE HERE

# # Adjust the columns names to the Prophet syntax
# mercado_sales_prophet_df.columns = # YOUR CODE HERE

# # Visualize the DataFrame
# # YOUR CODE HERE


In [ ]:
# # Create the model
# mercado_sales_prophet_model = # YOUR CODE HERE

# # Fit the model
# # YOUR CODE HERE


In [ ]:
# # Predict sales for 90 days (1 quarter) out into the future.

# # Start by making a future dataframe
# mercado_sales_prophet_future = # YOUR CODE HERE

# # Display the last five rows of the future DataFrame
# # YOUR CODE HERE


In [ ]:
# # Make predictions for the sales each day over the next quarter
# mercado_sales_prophet_forecast = # YOUR CODE HERE

# # Display the first 5 rows of the resulting DataFrame
# # YOUR CODE HERE


#### Step 2: Interpret the model output to identify any seasonal patterns in the company's revenue. For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

In [ ]:
# Use the plot_components function to analyze seasonal patterns in the company's revenue
# YOUR CODE HERE


##### Answer the following question:

**Question:** For example, what are the peak revenue days? (Mondays? Fridays? Something else?)

**Answer:** # YOUR ANSWER HERE

#### Step 3: Produce a sales forecast for the finance group. Give them a number for the expected total sales in the next quarter. Include the best- and worst-case scenarios to help them make better plans.

In [ ]:
# Plot the predictions for the Mercado sales
# YOUR CODE HERE


In [ ]:
# # For the mercado_sales_prophet_forecast DataFrame, set the ds column as the DataFrame Index
# mercado_sales_prophet_forecast = # YOUR CODE HERE

# # Display the first and last five rows of the DataFrame
# # YOUR CODE HERE


In [ ]:
# # Produce a sales forecast for the finance division
# # giving them a number for expected total sales next quarter.
# # Provide best case (yhat_upper), worst case (yhat_lower), and most likely (yhat) scenarios.

# # Create a forecast_quarter Dataframe for the period 2020-07-01 to 2020-09-30
# # The DataFrame should include the columns yhat_upper, yhat_lower, and yhat
# mercado_sales_forecast_quarter = # YOUR CODE HERE

# # Update the column names for the forecast_quarter DataFrame
# # to match what the finance division is looking for 
# mercado_sales_forecast_quarter = # YOUR CODE HERE

# # Review the last five rows of the DataFrame
# # YOUR CODE HERE


In [ ]:
# Displayed the summed values for all the rows in the forecast_quarter DataFrame
# YOUR CODE HERE


### Based on the forecast information generated above, produce a sales forecast for the finance division, giving them a number for expected total sales next quarter. Include best and worst case scenarios, to better help the finance team plan.

**Answer:** # YOUR ANSWER HERE